In [1]:
#造房子数据
import pandas as pd
import numpy as np
import random
from datetime import datetime

def generate_housing_data(num_samples=800):
    """
    生成房屋数据集
    
    参数:
    num_samples (int): 生成的样本数量，默认为800
    
    返回:
    pandas.DataFrame: 包含房屋特征和价格的DataFrame
    """
    # 设置随机种子以确保结果可重现
    np.random.seed(42)
    random.seed(42)
    
    # 房屋特征列表
    data = {
        '房屋ID': range(1, num_samples + 1),
        '建筑面积(平方米)': np.round(np.random.normal(120, 40, num_samples), 1),
        '卧室数量': np.random.randint(1, 6, num_samples),
        '卫生间数量': np.random.randint(1, 4, num_samples),
        '楼层数': np.random.randint(1, 4, num_samples),
        '建造年份': np.random.randint(1960, 2023, num_samples),
        '地理位置评分': np.random.randint(1, 11, num_samples),
        '距市中心距离(公里)': np.round(np.random.uniform(1, 30, num_samples), 1),
        '是否有车库': np.random.choice(['是', '否'], num_samples, p=[0.7, 0.3]),
        '装修等级': np.random.choice(['简装', '中装', '精装', '豪装'], num_samples, p=[0.2, 0.5, 0.2, 0.1]),
        '周边学校数量': np.random.randint(0, 6, num_samples)
    }
    
    # 创建DataFrame
    df = pd.DataFrame(data)
    
    # 确保建筑面积为正数
    df['建筑面积(平方米)'] = df['建筑面积(平方米)'].abs()
    
    # 计算房价（基于特征的线性组合加上一些噪声）
    # 房价基础公式（单位：万元）
    base_price = (
        df['建筑面积(平方米)'] * 0.8 +  # 每平米0.8万元
        df['卧室数量'] * 20 +
        df['卫生间数量'] * 15 +
        df['楼层数'] * 5 +
        (2023 - df['建造年份']) * (-0.5) +  # 越新的房子越贵
        df['地理位置评分'] * 10 +
        df['距市中心距离(公里)'] * (-2) +  # 距离越远越便宜
        (df['是否有车库'] == '是') * 15 +
        df['周边学校数量'] * 8
    )
    
    # 为装修等级分配不同的价值
    renovation_mapping = {'简装': 0, '中装': 20, '精装': 40, '豪装': 80}
    df['装修价值'] = df['装修等级'].map(renovation_mapping)
    base_price += df['装修价值']
    
    # 添加随机噪声（±10%）
    noise = np.random.normal(0, 0.1, num_samples)
    prices = base_price * (1 + noise)
    
    # 确保价格为正值并四舍五入到两位小数
    df['房价(万元)'] = np.round(np.abs(prices), 2)
    
    # 删除临时的装修价值列
    df.drop('装修价值', axis=1, inplace=True)
    
    return df

def save_to_excel(df, filename=None):
    """
    将DataFrame保存为Excel文件
    
    参数:
    df (pandas.DataFrame): 要保存的数据
    filename (str): 文件名，如果为None则使用时间戳生成
    """
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'房屋数据_{timestamp}.xlsx'
    
    # 使用ExcelWriter设置格式
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='房屋数据', index=False)
        
        # 获取工作簿和工作表对象用于设置格式
        workbook = writer.book
        worksheet = writer.sheets['房屋数据']
        
        # 设置列宽
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 20)
            worksheet.column_dimensions[column_letter].width = adjusted_width
    
    print(f"数据已保存到文件: {filename}")
    print(f"共生成 {len(df)} 条房屋记录")

def main():
    """主函数"""
    print("开始生成房屋数据...")
    
    # 生成800条房屋数据
    housing_data = generate_housing_data(800)
    
    # 显示数据基本信息
    print("\n数据概览:")
    print(housing_data.head())
    
    print("\n数据统计信息:")
    print(housing_data.describe())
    
    # 保存到Excel
    filename = "房屋数据集_800条.xlsx"
    save_to_excel(housing_data, filename)
    
    # 显示一些统计信息
    print(f"\n房价统计:")
    print(f"平均房价: {housing_data['房价(万元)'].mean():.2f} 万元")
    print(f"最低房价: {housing_data['房价(万元)'].min():.2f} 万元")
    print(f"最高房价: {housing_data['房价(万元)'].max():.2f} 万元")
    print(f"房价标准差: {housing_data['房价(万元)'].std():.2f} 万元")

if __name__ == "__main__":
    main()

开始生成房屋数据...

数据概览:
   房屋ID  建筑面积(平方米)  卧室数量  卫生间数量  楼层数  建造年份  地理位置评分  距市中心距离(公里) 是否有车库 装修等级  \
0     1      139.9     2      1    2  1964      10         8.0     是   中装   
1     2      114.5     3      2    3  2015       1         5.2     是   精装   
2     3      145.9     2      3    2  1991       9         4.4     是   简装   
3     4      180.9     1      1    3  1979       6         1.8     是   简装   
4     5      110.6     3      1    1  2009       9        16.2     否   中装   

   周边学校数量  房价(万元)  
0       2  236.96  
1       1  237.40  
2       5  271.66  
3       1  265.66  
4       3  295.32  

数据统计信息:
           房屋ID   建筑面积(平方米)        卧室数量       卫生间数量         楼层数         建造年份  \
count  800.0000  800.000000  800.000000  800.000000  800.000000   800.000000   
mean   400.5000  119.709125    3.022500    1.991250    1.966250  1991.353750   
std    231.0844   39.271363    1.429877    0.809007    0.835754    18.390184   
min      1.0000    9.700000    1.000000    1.000000    1.000000  1960